In [0]:
!pip install catboost

    100% |████████████████████████████████| 55.3MB 603kB/s 


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1zo_19GCSahf4WmlTH7468OW3oill8Ffi'
downloaded = drive.CreateFile({'id': file_id})
#输入你的文件名字
downloaded.GetContentFile('train.csv')
print('Downloaded content "{}"'.format(downloaded.GetContentString()))
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1bIAQ-9l_VvLb2dtUaF_n470m77r4_0Sm'
downloaded = drive.CreateFile({'id': file_id})
#输入你的文件名字
downloaded.GetContentFile('test.csv')
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S
13,0,3,"Saundercock, Mr. William H

In [0]:
import catboost
import pandas as pd
train=pd.read_csv('train.csv')
print(train.head())
print(train.keys())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
In

In [0]:
import numpy as np

In [0]:
train_null=train.isnull().sum(axis=0)
print(train_null)
print(train_null[train_null!=0])

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Age         177
Cabin       687
Embarked      2
dtype: int64


In [0]:
train.fillna(-999,inplace=True)

In [0]:
print(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [0]:
y=train['Survived']
X=train.drop('Survived',axis=1)

In [0]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [0]:
from catboost import Pool,cv,CatBoostClassifier
from sklearn.metrics import accuracy_score

In [0]:
model=CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent'
)

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=42)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
model.fit(
    X_train,y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_test,y_test),
    #plot=True
)

In [0]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

In [0]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, best_model.predict(X_test))
))

Simple model validation accuracy: 0.8251

Best model validation accuracy: 0.8386


In [0]:
!pip install hyperopt

    100% |████████████████████████████████| 122kB 4.1MB/s 
    100% |████████████████████████████████| 409kB 5.4MB/s 


In [0]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric='Accuracy',
        random_seed=42,
        logging_level='Silent'
    )
    
    cv_data = cv(
        Pool(X, y, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [0]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

{'l2_leaf_reg': 5.0, 'learning_rate': 0.1147638000846512}


In [0]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='Accuracy',
    random_seed=42,
    logging_level='Silent'
)

In [0]:
model.fit(X, y, cat_features=categorical_features_indices)

In [0]:
import pandas as pd
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model.predict(X_test)

In [0]:
print(X_test.head())
print(test_X.head())

     PassengerId  Pclass                                               Name  \
709          710       3  Moubarek, Master. Halim Gonios ("William George")   
439          440       2             Kvillner, Mr. Johan Henrik Johannesson   
840          841       3                        Alhomaki, Mr. Ilmari Rudolf   
720          721       2                  Harper, Miss. Annie Jessie "Nina"   
39            40       3                        Nicola-Yarred, Miss. Jamila   

        Sex    Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
709    male -999.0      1      1              2661  15.2458  -999        C  
439    male   31.0      0      0        C.A. 18723  10.5000  -999        S  
840    male   20.0      0      0  SOTON/O2 3101287   7.9250  -999        S  
720  female    6.0      0      1            248727  33.0000  -999        S  
39   female   14.0      1      0              2651  11.2417  -999        C  
   PassengerId  Pclass                                         

In [0]:
import pandas as pd
test_X=pd.read_csv('test.csv')
test_X.fillna(-999,inplace=True)
test_y=model.predict(test_X)
submisstion = pd.DataFrame()
submisstion['PassengerId']=test_X['PassengerId']
submisstion['Survived']=test_y


In [0]:
submisstion.to_csv('submission.csv', index=False)
files.download('submission.csv')

In [0]:
df_test=pd.read_csv('test.csv')
test_y=test_y.astype(np.int64)
df_test['Survived']=test_y.tolist()
df_result=df_test.loc[:,['PassengerId','Survived']]

from google.colab import files
with open('result.csv', 'w') as f:
  f.write('')
df_result.to_csv('result.csv',index=False)

files.download('result.csv')

In [0]:
from google.colab import files
with open('submission.csv', 'w') as f:
  f.write('')
submisstion.to_csv('submission.csv',index=False)

files.download('submission.csv')

In [0]:
print(df_result.dtypes)

PassengerId    int64
Survived       int64
dtype: object


In [0]:

#modelmodel  ==  CatBoostClassifierCatBoost (iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
files.download('catboost_model.dump')